In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import DataLoader
import os
import time
from Common import NeuralNet, fit
torch.manual_seed(42)


## Multilevel Training for the Heat Equation

Consider the parametric one-dimensional heat equation:

$$
u_t(t, x, y) = ku_{xx}(t, x, y), \quad x\in [0,1], t\in[0,T], y\in[0,1]^d
$$

with zero Dirichlet boundary conditions and parametrized initial condition
$$
u_0(x,y) = \sum_{j=1}^d (2y_j -1 ) \sin(2\pi j x)
$$
The observable is the heat flux at (T,0):
$$
L(y) = -k\frac{\partial T}{\partial x}|_{t=T,x=0}
$$

The heat equation can be solved with a simple finite difference scheme given a mesh with grid sizes $\Delta x$ and $\Delta t$, and the heat flux computed from the resulting numerical approximation. 
Let us define the approximate flux as $L^\Delta$.

We are interested in the input to the observable map:
$$
L: y \mapsto L^\Delta(y), \quad y\in[0,1]^d
$$



In [ ]:
def solve_heat_eq(x, y):
    T = 0.01
    ic = (2 * y[0] - 1) * np.sin(2 * np.pi * x) + \
         (2 * y[1] - 1) * np.sin(4 * np.pi * x) + \
         (2 * y[2] - 1) * np.sin(6 * np.pi * x) + \
         (2 * y[3] - 1) * np.sin(8 * np.pi * x)

    
    # Heat Diffusivity
    diff = 1
    # Grid spacing delta_x
    dx = x[1] - x[0]
    # Time step delta_t
    dt = 0.4 * dx ** 2 / diff

    F = diff * dt / dx ** 2
    nt = int((T / dt))
    nx = x.shape[0]

    u_old = ic
    u_new = np.zeros_like(ic)

    for k in range(1, nt):
        for i in range(1, nx - 1):
            u_new[i] = u_old[i] + F * (u_old[i + 1] - 2 * u_old[i] + u_old[i - 1])
        u_new[0] = 0
        u_new[-1] = 0
        u_old[:] = u_new

    # Compute the heat flux at x=0, t = T
    flux = -diff * (u_new[0] - u_new[1]) / dx
    return flux



Consider a hierarchy of meshes with grid size $\Delta_\ell =\frac{\Delta_0}{2^\ell}$, $\ell=0,...,L$, $\Delta_0 = \frac{1}{20}$ and express the observable $L^{\Delta_L}(y)$ at the finest mesh resolution $\Delta_L$ in terms of a telescopic sum:

$$
L^{\Delta_L}(y) = L^{\Delta_0}(y) + \sum_{\ell=1}^L (L^{\Delta_\ell}(y) - L^{\Delta_{\ell-1}}(y))= L^{\Delta_0}(y) + \sum_{\ell=1}^L D_\ell(y)
$$ 
with

$$
D_\ell (y) = L^{\Delta_\ell}(y) - L^{\Delta_{\ell-1}}(y),\quad \ell=1,...,L 
$$

denoted as details.

**Idea multilevel training**: instead of approximating directly the map $y \mapsto L^{\Delta_L}(y)$ with neural network $L^{\Delta_L, \ast} \approx L^{\Delta_L}$ trained with a training set $S_L = \{ (y_k, L^{\Delta_L}_k),\quad k=1,...,N_L\}$, define the approximate model as:

$$
 L^{\Delta_L, \ast}(y) = L^{\Delta_0, \ast}(y) + \sum_{\ell=1}^L D^\ast_\ell(y)
$$

with

$$
L^{\Delta_0, \ast}\approx L^{\Delta_0}, \quad D^\ast_\ell \approx D_\ell ~~\ell=1,...,L
$$

being neural networks trained with training sets $S_0= \{ (y_k, L^{\Delta_0}_k), k=1,...,N_0\}$, $S_\ell= \{ (y_k, D_{\ell,k}), k=1,...,N_L\}$ and

$$
N_\ell = N_L\cdot 2^{L-\ell}.
$$

Observe that the k-th realization of the detail $D_{\ell,k} $ has to be computed given the same realization of the input parameter $y_k$. 


**Generate $N_\ell$, $\ell=0,...,L$, realizations of the observable at different mesh resolution to assemble later on the training sets**

In [ ]:
print("###############################")
print("Generating Training Sets")
np.random.seed(42)
L=3                                                # Finest resolution level L
l_values = np.array([L-3,L-2,L-1,L])               # List of all resolution levels l, from the finest (l=L=3) to the coarsest (l=0)

ns_finest=50                                       # Number of training samples at the finest resolution level L
nx_coarsest = 20                                   # Number of grid points at the coarsest resolution level L
nx_values = nx_coarsest*2**l_values                # Number of grid points at the remaining resolution levels
nx_finest = nx_values[-1]

ns_values = ns_finest*2**(l_values[-1] - l_values) # Number of training samples at the remaining resolution levels

# Generate N_0 realizations of the parameters
y = np.random.random((ns_values[0], 4))

datasets_meshes = []
total_time = 0
# Loop over the meshes and generate ns realizations of the observable at the mesh with nx grid points 
for nx, ns in zip(nx_values, ns_values):

    s = np.zeros((ns, y.shape[1] + 1))

    print("##############################################################")
   
    x_ = np.linspace(0, 1, nx + 1)
    start = time.time()
    for j in range(ns):
        f = solve_heat_eq(x_, y[j])
        s[j, :4] = y[j]
        s[j, -1] = f
    end = round((time.time() - start)/ns, 4)
    total_time = total_time + end*ns
    print("Generated ", ns," realizations of the observable on a spatial mesh with ", nx + 1, " grid points. Time per sample: ", end)

    datasets_meshes.append(s)
print("Total time required to generate data: ", total_time)

###############################
Generating Training Sets
##############################################################
Generated  400  realizations of the observable on a spatial mesh with  21  grid points. Time per sample:  0.0002
##############################################################
Generated  200  realizations of the observable on a spatial mesh with  41  grid points. Time per sample:  0.0017
##############################################################
Generated  100  realizations of the observable on a spatial mesh with  81  grid points. Time per sample:  0.0135
##############################################################
Generated  50  realizations of the observable on a spatial mesh with  161  grid points. Time per sample:  0.1069
Total time required to generate data:  7.115


**Generate training set $S_\ell$ for learning the detail maps $y\mapsto D_\ell(y)$, $\ell=1,...,L$**

In [ ]:
training_sets = list()
training_sets.append(datasets_meshes[0])

# Obtain the details at different mesh resolutions 
for l in range(1, len(datasets_meshes)):
    ns = datasets_meshes[l].shape[0]
    assert ((datasets_meshes[l][:ns, :y.shape[1]] == datasets_meshes[l - 1][:ns, :y.shape[1]]).all())

    obs_diff = datasets_meshes[l][:ns, -1] - datasets_meshes[l - 1][:ns, -1]
    print("Variance of details at level l =", l ,": ",np.var(obs_diff))
    ts_detail_l = np.concatenate([datasets_meshes[l][:ns, :y.shape[1]], obs_diff.reshape(-1, 1)], 1)
    training_sets.append(ts_detail_l)

Variance of details at level l = 1 :  0.07059189551141742
Variance of details at level l = 2 :  0.0038554830215633256
Variance of details at level l = 3 :  0.00019470864320350647


**Train all the building models of the multilevel scheme**
$$
L^{\Delta_0, \ast}\approx L^{\Delta_0}, \quad D^\ast_\ell \approx D_\ell, ~~\ell=1,...,L
$$

In [ ]:
approximate_models = list()

for i, current_ts in enumerate(training_sets):
    inputs = torch.from_numpy(current_ts[:, :y.shape[1]]).type(torch.float32)
    output = torch.from_numpy(current_ts[:, -1].reshape(-1, 1)).type(torch.float32)
    batch_size = inputs.shape[0]
    training_set = DataLoader(torch.utils.data.TensorDataset(inputs, output), batch_size=batch_size, shuffle=True)

    model = NeuralNet(input_dimension=inputs.shape[1], 
                      output_dimension=output.shape[1], 
                      n_hidden_layers=4, 
                      neurons=20,
                      regularization_param=0.0, 
                      regularization_exp=2, 
                      retrain_seed=128)
    

    optimizer_ = optim.LBFGS(model.parameters(), lr=0.1, max_iter=1, max_eval=50000, tolerance_change=1.0 * np.finfo(float).eps)

    n_epochs = 500
    history = fit(model, training_set, n_epochs, optimizer_, p=2, verbose=False)
    print("Final Training loss: ", history[-1])


    approximate_models.append(model)

Final Training loss:  3.265416307840496e-05
Final Training loss:  2.2737967810826376e-06
Final Training loss:  6.867440447422268e-07
Final Training loss:  3.5353078686739536e-08


In [ ]:
def predcit_with_ml(list_models, inputs_):
    output_ = torch.zeros((inputs_.shape[0], 1))
    for i in range(len(list_models)):
        output_ = output_ + list_models[i](inputs_)
    return output_


**Generate $N_{sl}$ realizations of the observable at the finest mesh resolution to equate the total cost required by the multilevel scheme**

In [ ]:
training_data_single_level = list()
start = time.time()
current_elapsed_time = 0
x_ = np.linspace(0, 1, nx_finest)
j = 0
while current_elapsed_time < total_time:
    f = solve_heat_eq(x_, y[j])
    sample = np.concatenate([y[j], np.array(f).reshape(1,)]).reshape(1,-1)
    training_data_single_level.append(sample)
    current_elapsed_time = time.time() - start
    j=j+1
    
print("Generated ", len(training_data_single_level), " on the finest mesh. Total time: ", current_elapsed_time)
training_data_single_level = np.concatenate(training_data_single_level)

Generated  67  on the finest mesh. Total time:  7.1806230545043945


**Train the corresponding single-level model by using $N_{sl}$ realization of the observable at the finest mesh resolution**

In [ ]:
inputs = torch.from_numpy(training_data_single_level[:, :y.shape[1]]).type(torch.float32)
output = torch.from_numpy(training_data_single_level[:, -1].reshape(-1, 1)).type(torch.float32)
batch_size = inputs.shape[0]
training_set = DataLoader(torch.utils.data.TensorDataset(inputs, output), batch_size=batch_size, shuffle=True)

model_finest = NeuralNet(input_dimension=inputs.shape[1], 
                         output_dimension=output.shape[1], 
                         n_hidden_layers=4, 
                         neurons=20, 
                         regularization_param=0.0, 
                         regularization_exp=2,
                         retrain_seed=128)
optimizer_ = optim.LBFGS(model_finest.parameters(), lr=0.1, max_iter=1, max_eval=50000, tolerance_change=1.0 * np.finfo(float).eps)

n_epochs = 500
history = fit(model_finest, training_set, n_epochs, optimizer_, p=2, verbose=False)
print("Final Training loss: ", history[-1])


Final Training loss:  1.1044460734410677e-05


**Generate test set (at finest mesh resolution) and assess models performance**

In [ ]:
def generate_test_set(n_samples, n_grid):
    np.random.seed(34)
    inputs_ = np.random.random((n_samples, 4))
    s_ = np.zeros((n_samples, inputs_.shape[1] + 1))
    x_ = np.linspace(0, 1, n_grid)
    for j in range(n_samples):
        s_[j, :4] = inputs_[j]
        s_[j, -1] = solve_heat_eq(x_, inputs_[j])

    return s_

test_set = generate_test_set(1024, nx_finest)

In [ ]:
test_inputs = torch.from_numpy(test_set[:, :4]).type(torch.float32)
test_output = torch.from_numpy(test_set[:, -1]).type(torch.float32).reshape(-1, )

test_pred_ml = predcit_with_ml(approximate_models, test_inputs).reshape(-1, )
test_pred_finest = model_finest(test_inputs).reshape(-1, )
err_ml = (torch.mean((test_output - test_pred_ml) ** 2) / torch.mean(test_output ** 2)) ** 0.5
err_fin = (torch.mean((test_output - test_pred_finest) ** 2) / torch.mean(test_output ** 2)) ** 0.5

print("Error Multilevel model: ", err_ml.item())
print("Error Singlelevel model: ", err_fin.item())

Error Multilevel model:  0.010485476814210415
Error Singlelevel model:  0.020693715661764145
